config는 실험용 데이터의 path를 저장한 파일이므로, 여러분의 코퍼스의 위치를 지정하시면 됩니다. 

In [1]:
from config import raw_corpus_fname, tokenized_corpus_fname
print('raw_corpus_fname과 tokenized_corpus_fname의 타입은 str입니다. ')
print(type(raw_corpus_fname), type(tokenized_corpus_fname))

import sys
sys.path.insert(0, '../')
import soykeyword

raw_corpus_fname과 tokenized_corpus_fname의 타입은 str입니다. 
<class 'str'> <class 'str'>


In [2]:
tokenized_corpus_fname.split('/')[-1]

'2016-10-20_article_all_normed_nountokenized.txt'

Proportion ratio를 이용하는 키워드 추출 방법은 CorpusbasedKeywordExtractor와 MatrixbasedKeywordExtractor 두 종류로 구현되어 있습니다. CorpusbasedKeywordExtractor는 아래 예시처럼 text 파일에서 키워드를 직접 추출하는 코드이며, MatrixbasedKeywordExtractor는 sparse format으로 만들어둔 term frequency matrix에서 키워드를 추출하는 코드입니다. 

Proportion ratio의 개념은 아래와 같습니다. 

키워드란 사실 명확한 정의가 있는 단어가 아닙니다. 흔히 사용하는 키워드의 정의에는 자주 나오는 단어나, TF-IDF 값이 있습니다. 이들은 각자의 관점으로 키워드를 정의한 것입니다. 자주 나오는 단어를 키워드로 정의하는 것은 많이 나올수록 키워드라는 의미이며, 이 때에는 조사와 같은 단어가 키워드가 될 수도 있습니다. 이를 보완하기 위해 품사 판별 (Part-of-Speech tagging)을 한 뒤, 명사만을 추출하여 최빈어를 키워드로 선정하는 것은 합리적이라 생각됩니다. TF-IDF를 키워드로 사용하는 방법은 조금 위험한 방법입니다. IDF는 단어가 등장한 문서의 개수가 적을수록 커지기 때문에 오히려 노이즈일 가능성이 높기 때문입니다. 

그 외에도 chi-square를 이용하는 방법도 있습니다. 제가 제안하는 Proportion ratio는 이 방법에 가깝습니다. 기본 컨셉은 아래와 같습니다. '뉴스에서의 키워드'를 선택하라는 말은 애매모호합니다. 하지만, '오늘의 뉴스에서의 키워드'나 '아이오아이에 대한 문서에서의 키워드'라는 말은 조금 더 명료합니다. 키워드에 대한 관점이 생기기 때문입니다. 좀 더 자세히, 여름철 뉴스에서는 '호우'라는 단어가 0.1% 씩 늘 등장한다고 가정합시다. 어느날 '호우'라는 단어가 평상시와 다르게 0.9% 등장하였다면 (평상시보다 9배), 이 날은 정말로 호우가 내려서 뉴스에 그 단어가 자주 등장했을 가능성이 높습니다. 그렇다면 '호우'는 그날의 키워드가 될 수 있을 것입니다. 이를 수치로 만들기 위해서 다음과 같은 지표를 만들었습니다. 

    score(w) = P(w|Dt) / { P(w|Dt) + P(w|Dr) }
    
    P(w|Dt): target document에서 단어 w가 출현한 비율
    P(w|Dr): reference document에서 단어 w가 출현한 비율

Target document란 키워드를 정의하고 싶은 문서 집합을 의미합니다. '아이오아이라는 단어가 포함된 뉴스'라던가, 어느날의 뉴스가 됩니다. Reference document는 평상시의 문서 집합입니다. '아이오아이'를 포함한 연예뉴스 가 될 수도, 하루치 전체 뉴스가 될 수도 있습니다. 어느 하루의 뉴스의 키워드를 선택하기 위해서는 이전 10일치의 뉴스를 reference document로 선택할 수 있습니다. 

이렇게 keyword score를 정의하면 score(w)는 [0, 1] 사이의 값이 됩니다. 평상시 호우가 0.1% 등장하였다가 오늘 0.9% 등장하였다면 score는 0.9 / (0.1 + 0.9) = 0.9 입니다. 평상시와 같이 0.1% 등장하였다면 (0.1 / (0.1 + 0.1)) = 0.5가 됩니다. 0.5란 평상시와 다르지 않다는 의미이며, 그 이하는 평상시보다 등장하지 않았다는 의미입니다. 하지만 0.5보다 작은 값은 의미가 없습니다. target document set은 reference document set보다 훨씬 작은 집합이기 때문에 많은 단어를 포함하지 않을 수 있기 때문입니다. 대신 0.5보다 큰 score를 지니는 단어들은 평상시보다 자주 등장한 단어임을 의미합니다. 이때에는 한가지 false alarm이 생길 수 있습니다. 애초에 자주 등장하지 않는 단어이기 때문에 target documents에만 등장하는 단어는 1.0에 가까운 score를 가지게 됩니다. 이를 방지하기 위해서 최소한 등장해야 하는 단어 빈도수를 한정할 필요가 있습니다. 그래서 키워드를 선택할 때 항상 parameter로 min_frequency를 넣도록 하였습니다. 

    keywords = corpusbased_extractor.extract_from_word('아이오아이', min_score=0.8, min_frequency=100)

In [3]:
class Corpus:
    def __init__(self, fname):
        self.fname = fname
        self.length = 0
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip()
    def __len__(self):
        if self.length == 0:
            with open(self.fname, encoding='utf-8') as f:
                for n_doc, _ in enumerate(f):
                    continue
                self.length = (n_doc + 1)
        return self.length

tutorial에 올리는 파일은 2016-10-20의 하루치 뉴스를 크롤링한 데이터입니다. 데이터는 공유할 수 없음을 양해 부탁드립니다. 사용하는 데이터 포멧은 한 줄이 하나의 뉴스에 해당합니다. 하루치 뉴스에 대하여 명사를 추출한 뒤, 이를 list 형태로 저장하고 있습니다. doc.split()을 하면 명사들이 return 됩니다. 

In [4]:
for i, doc in enumerate(Corpus(tokenized_corpus_fname)):
    if i <= 5: continue
    if i > 10: break
    print('doc=%d, num words=%d, %s\n' % (i, len(doc.split()), doc[:200]))

doc=6, num words=187, 1억 달러 리스크 강화 뉴욕 연합뉴스 특파원 미국 대형은행 골드만삭스 근무 트레이더 투자 등급 채권 정크본드 투자 사이 1억 달러 1천 이익 것으로 금융 글로벌 금융위기 이후 대형은행 리스크 강조 상황 이런 대박 않은 것으로 월스트리트저널 뉴욕 고수익 골드만삭스 34 관리 정크본드 투자 1억 달러 이상 수익 골드만삭스 소식통 인용 19일 현지시간 보도 1월

doc=7, num words=25, 서울 연합뉴스 19일 서울 시내 출동 경찰관 사제 총기 발사 용의자 범행 사회관계망서비스 용의자 경찰 내게 살인 누명 경찰 적대감 차례 2016 페이스북 캡처 연합뉴스

doc=8, num words=271, 제안 한미 해군 수상전센터 미래 해상 연구 워싱턴 연합뉴스 이영 기자 제4 한미안보협의회 참석 미국 방문 한민구 국방부 장관 19일 현지시간 해군 최첨단 무기체계 개발 수상전센터 방문 국방부 관계자는 이날 장관 미국 버지니아주 해군 수상전센터 무기체계 개발 현황 한국 국방부 장관 방문 이번 처음 장관 방문 미국 제안 것으로 장관 한미 외교 국방장관 회의 참

doc=9, num words=32, 서울 연합뉴스 기자 19일 서울 시내 경찰관 사제총기 발사 경찰관 숨지게 폭행 용의자 인근 상인들 것으로 확인 왼쪽 사진 신발 주인 직원 오른쪽 사진 매운탕 주인 시민 3명 경찰 용의자 검거 2016

doc=10, num words=134, 서울경찰청장 병원 방문 유족들 위로 서울 연합뉴스 박경 기자 19일 사제 총기범 총탄 김창 54 경위 시신이 안치 서울 도봉구 한일병원 유족들 슬픔 감추지 유족들 안치 경위 시신 보고 오열 외아들 아내 경위 침상 떠나지 부인 오열 쓰러져 의료진 치료 받기 것으로 병원 유가족 물론 동료 경찰들 소식 달려 애도 동료 경찰들 평소 의협심 후임 먼저 경찰 경위 사



CorpusbasedKeywordExtractor를 만들 때, 애초에 키워드 후보가 될 수 있는 단어를 minimum term frequency (min_tf)와 minimum document frequency (min_df)로 필터링 할 수 있도록 하였습니다. 키워드의 후보들은 모두 min_tf, min_df 이상이 되는 단어들로 한정됩니다. 

tokenize는 텍스트 형식의 corpus에서 단어를 추출하기 위한 tokenizer입니다. 기본값은 띄어쓰기입니다만, KoNLPy의 nouns()나 pos()를 이용할 수도 있습니다. 

In [5]:
from soykeyword.proportion import CorpusbasedKeywordExtractor

corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    tokenize=lambda x:x.strip().split(),
    verbose=True
)

corpusbased_extractor.train(Corpus(tokenized_corpus_fname))

training was done 34572 terms, 30091 docs, memory = 0.438 Gb6 Gb


각 단어가 corpus에서 몇 번 등장했는지 빈도를 알 수 있습니다. 

In [6]:
for word in ['박근혜', '문재인', '최순실', '아이오아이', '트와이스', '군사', '외교']:
    print(word, corpusbased_extractor.frequency(word))

박근혜 1445
문재인 1010
최순실 1318
아이오아이 270
트와이스 655
군사 170
외교 881


존재하지 않는 단어는 빈도수가 0으로 출력됩니다. 

In [7]:
corpusbased_extractor.frequency('lovit')

0

'아이오아이'가 포함된 문서 번호 (텍스트 파일에서의 line number)를 가지고 올 수도 있습니다. 

In [8]:
documents = corpusbased_extractor.get_document_index('아이오아이')
documents[:10]

[6884, 6897, 6956, 7338, 7345, 7582, 8011, 8053, 9180, 9228]

키워드를 선택하는 방법은 아래와 같이 두가지 입니다. 

    corpusbased_extractor.extract_from_word('아이오아이', min_score=0.8, min_frequency=100)
    corpusbased_extractor.extract_from_docs(documents, min_score=0.8, min_frequency=100)
    
extract_from_word(aspect_word)는 기준점이 되는 단어를 넣으면 min_score, min_frequency 이상이 되는 단어들 을 선택합니다. target document는 aspect_word가 포함된 문서 집합이며, reference document는 aspect_word가 포함되지 않은 문서 집합입니다. 

'아이오아이'가 포함된 문서가 target document이기 때문에 '아이오아이' 단어는 score가 반드시 1.0입니다. 그 외의 키워드에서 '엠카운트다운'이나 '걸크러쉬', '타이틀곡' 등이 키워드로 선택된 것으로 보아 Mnet의 엠카운트다운에 출현한 내용에 대한 기사들이 있던 것으로 추정됩니다. 

In [9]:
keywords = corpusbased_extractor.extract_from_word(
    '아이오아이',
    min_score=0.8,
    min_frequency=100
)

keywords[:10]

[KeywordScore(word='아이오아이', frequency=270, score=1.0),
 KeywordScore(word='엠카운트다운', frequency=221, score=0.997897148491129),
 KeywordScore(word='펜타곤', frequency=104, score=0.9936420169665052),
 KeywordScore(word='잠깐', frequency=162, score=0.9931809154109712),
 KeywordScore(word='엠넷', frequency=125, score=0.9910325251765126),
 KeywordScore(word='걸크러쉬', frequency=111, score=0.9904705029926091),
 KeywordScore(word='타이틀곡', frequency=311, score=0.987384461584851),
 KeywordScore(word='코드', frequency=105, score=0.9871835929954923),
 KeywordScore(word='본명', frequency=105, score=0.9863934667369743),
 KeywordScore(word='엑스', frequency=101, score=0.9852544036088814)]

혹은 직접 target document의 document index를 입력할 수도 있습니다. 앞서서 '아이오아이'가 포함된 문서아이디를 documents로 선택하였기 때문에 아래의 키워드 추출 결과는 위와 동일합니다. 

In [10]:
keywords = corpusbased_extractor.extract_from_docs(
    documents,
    min_score=0.8,
    min_frequency=100
)

keywords[:10]

[KeywordScore(word='아이오아이', frequency=270, score=1.0),
 KeywordScore(word='엠카운트다운', frequency=221, score=0.997897148491129),
 KeywordScore(word='펜타곤', frequency=104, score=0.9936420169665052),
 KeywordScore(word='잠깐', frequency=162, score=0.9931809154109712),
 KeywordScore(word='엠넷', frequency=125, score=0.9910325251765126),
 KeywordScore(word='걸크러쉬', frequency=111, score=0.9904705029926091),
 KeywordScore(word='타이틀곡', frequency=311, score=0.987384461584851),
 KeywordScore(word='코드', frequency=105, score=0.9871835929954923),
 KeywordScore(word='본명', frequency=105, score=0.9863934667369743),
 KeywordScore(word='엑스', frequency=101, score=0.9852544036088814)]

그 외의 단어들에 대해서도 키워드를 선택하면 아래와 같습니다. 아래는 키워드를 <단어, (빈도수, score)>로 표현한 것들입니다. 2016-10-20에는 박근혜게이트가 언론에 보도되기 시작하는 시기입니다. 당시에 비선실세와 연설문 사건들이 뉴스에 등장하기 시작했습니다. 이러한 단어들이 실제로 추출됨을 확인할 수 있습니다. 

우리는 extract_from_word(aspect_word)의 결과를 다르게 해석할 수 있습니다. 한 단어를 기준으로 target document를 잡은 뒤 키워드를 선택하는 것은 aspect_word가 들어간 문서를 잘 구분하는 features 이면서도 유독 aspect_word와 함께 등장하는 단어라는 뜻입니다. 즉, aspect_word의 연관어로 해석할 수도 있습니다. 

In [11]:
for word in ['박근혜', '문재인', '최순실', '아이오아이', '트와이스', '군사', '외교']:

    keywords = corpusbased_extractor.extract_from_word(
        word, min_score=0.8,
        min_frequency=150
    )

    keywords = keywords[:48]

    word_frequency = corpusbased_extractor.frequency(word)
    print('Aspect word = %s (%d)' % (word, word_frequency))

    def in_a_line(subkeywords):
        def tuple_to_strf(keyword):
            return '%s (%d, %.2f)' % keyword        
        strf = [tuple_to_strf(keyword) for keyword in subkeywords]
        strf = ['%17s' % s for s in strf]
        return '  -  '.join(strf)

    for i in range(12):
        subkeywords = keywords[4*i:4*i+4]
        line = in_a_line(subkeywords)
        print(line)

    print('-'*80)

Aspect word = 박근혜 (1445)
 박근혜 (1445, 1.00)  -  수석비서관회의 (208, 1.00)  -    재단들 (152, 1.00)  -    연설문 (204, 0.99)
  누구라 (178, 0.99)  -   불법행위 (240, 0.99)  -     퇴임 (188, 0.98)  -     엄정 (388, 0.98)
 창조경제 (226, 0.98)  -    처벌받 (227, 0.98)  -     미르 (604, 0.98)  -  스포츠재단 (676, 0.97)
더블루케이 (194, 0.97)  -     최씨 (695, 0.97)  -    재단 (1690, 0.97)  -  자유학기제 (201, 0.97)
 비선실세 (219, 0.97)  -   최순실씨 (520, 0.97)  -   미르재단 (247, 0.96)  -    게이트 (303, 0.96)
 대통령 (5682, 0.96)  -     모녀 (223, 0.96)  -   행복교육 (227, 0.95)  -     실세 (309, 0.95)
   비선 (288, 0.95)  -   최순실 (1318, 0.95)  -    의혹 (3602, 0.95)  -     고양 (278, 0.95)
   국정 (185, 0.94)  -   청와대 (2112, 0.94)  -    지지층 (151, 0.94)  -    킨텍스 (332, 0.94)
   체육 (221, 0.94)  -     재계 (152, 0.93)  -     민생 (164, 0.93)  -  2002년 (186, 0.93)
   정권 (596, 0.93)  -     가중 (175, 0.93)  -     유용 (359, 0.93)  -    전경련 (348, 0.93)
   주재 (459, 0.93)  -    국민들 (441, 0.93)  -     백승 (216, 0.92)  -    갤러리 (271, 0.92)
  기업들 (808, 0.92)  -    지지율 (336, 0.92)  -     확산

다른 머신러닝 알고리즘에 적용하기 위해서 sparse matrix 형식으로 데이터를 저장해둔 경우들도 있습니다. 이때에도 키워드 추출이 용이하도록 MatrixbasedKeywordExtractor를 만들어두었습니다. Interface나 작동 방식은 위와 동일합니다. 

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.001)
x = vectorizer.fit_transform(Corpus(tokenized_corpus_fname))

print(x.shape)

(30091, 9774)


scikit-learn의 CountVectorizer를 이용하여 term frequency matrix; x를 만들고, {word:index}의 dictionary와 [word, ...]의 list of str을 만들어두었습니다. 

In [13]:
word2index = vectorizer.vocabulary_
index2word = sorted(
    vectorizer.vocabulary_,
    key=lambda x:vectorizer.vocabulary_[x]
)

Matrix 형식이기 때문에 tokenize는 필요없습니다. 그 외의 parameters는 동일합니다. 

In [14]:
from soykeyword.proportion import MatrixbasedKeywordExtractor

matrixbased_extractor = MatrixbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    verbose=True
)

matrixbased_extractor.train(x)

MatrixbasedKeywordExtractor trained


matrixbased_extractor.train은 두 가지 argument를 필요로 합니다. 

    matrixbased_extractor.train(x, index2word=None)
    
만약 index2word가 입력되지 않으면 word index로만 키워드가 선택됩니다. 5537은 '아이오아이'입니다. matrixbased_extractor.train(5537)의 결과는 아래처럼 word index로 출력됩니다. 

In [15]:
keywords = matrixbased_extractor.extract_from_word(
    5537,
    min_score=0.8,
    min_frequency=100
)

keywords[:10]

[KeywordScore(word=5537, frequency=270, score=1.0),
 KeywordScore(word=5880, frequency=221, score=0.9978307775631691),
 KeywordScore(word=8976, frequency=104, score=0.9934422266805437),
 KeywordScore(word=7126, frequency=162, score=0.9929667382454291),
 KeywordScore(word=5879, frequency=125, score=0.9907514986652862),
 KeywordScore(word=1103, frequency=111, score=0.99017203825805),
 KeywordScore(word=8721, frequency=311, score=0.9869906112674688),
 KeywordScore(word=8651, frequency=105, score=0.9867835556082788),
 KeywordScore(word=4035, frequency=105, score=0.98596911773225),
 KeywordScore(word=5869, frequency=101, score=0.9847950780631249)]

이를 앞서 만든 index2word를 이용하여 decoding하면 아래와 같은 결과가 나옵니다. 이는 CorpusbasedKeywordExtractor의 결과와 같습니다. 

In [16]:
for keyword in keywords[:10]:

    word = index2word[keyword.word]
    frequency = keyword.frequency
    score = keyword.score

    print('word=%s, frequency=%d, score=%.3f' % (
        word, frequency, score))

word=아이오아이, frequency=270, score=1.000
word=엠카운트다운, frequency=221, score=0.998
word=펜타곤, frequency=104, score=0.993
word=잠깐, frequency=162, score=0.993
word=엠넷, frequency=125, score=0.991
word=걸크러쉬, frequency=111, score=0.990
word=타이틀곡, frequency=311, score=0.987
word=코드, frequency=105, score=0.987
word=본명, frequency=105, score=0.986
word=엑스, frequency=101, score=0.985


train()에서 index2word를 입력하지 않았기 때문에 extract_from_word()에 str 형식의 단어를 입력하면 index2word를 먼저 넣으라는 Exception이 발생합니다. 

In [17]:
try:
    keywords = matrixbased_extractor.extract_from_word(
        '아이오아이',
        min_score=0.8,
        min_frequency=100
    )
except Exception as e:
    print(e)

If you want to insert str word, you should trained index2word first


하지만 train단계에서 아래와 같이 index2word; list of str(word)를 넣어주면 5537과 같은 word index를 입력하여도, '아이오아이'와 같은 단어를 입력하여도 모두 아래와 같이 단어로 표현된 키워드 추출 결과가 return 됩니다. 

In [18]:
matrixbased_extractor_w_indexer = MatrixbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    verbose=True
)

matrixbased_extractor.train(x, index2word)

MatrixbasedKeywordExtractor trained


In [19]:
keywords = matrixbased_extractor.extract_from_word(
    5537,
    min_score=0.8,
    min_frequency=100
)

keywords[:10]

[KeywordScore(word='아이오아이', frequency=270, score=1.0),
 KeywordScore(word='엠카운트다운', frequency=221, score=0.9978307775631691),
 KeywordScore(word='펜타곤', frequency=104, score=0.9934422266805437),
 KeywordScore(word='잠깐', frequency=162, score=0.9929667382454291),
 KeywordScore(word='엠넷', frequency=125, score=0.9907514986652862),
 KeywordScore(word='걸크러쉬', frequency=111, score=0.99017203825805),
 KeywordScore(word='타이틀곡', frequency=311, score=0.9869906112674688),
 KeywordScore(word='코드', frequency=105, score=0.9867835556082788),
 KeywordScore(word='본명', frequency=105, score=0.98596911773225),
 KeywordScore(word='엑스', frequency=101, score=0.9847950780631249)]

In [20]:
keywords = matrixbased_extractor.extract_from_word(
    '아이오아이',
    min_score=0.8,
    min_frequency=100)

keywords[:10]

[KeywordScore(word='아이오아이', frequency=270, score=1.0),
 KeywordScore(word='엠카운트다운', frequency=221, score=0.9978307775631691),
 KeywordScore(word='펜타곤', frequency=104, score=0.9934422266805437),
 KeywordScore(word='잠깐', frequency=162, score=0.9929667382454291),
 KeywordScore(word='엠넷', frequency=125, score=0.9907514986652862),
 KeywordScore(word='걸크러쉬', frequency=111, score=0.99017203825805),
 KeywordScore(word='타이틀곡', frequency=311, score=0.9869906112674688),
 KeywordScore(word='코드', frequency=105, score=0.9867835556082788),
 KeywordScore(word='본명', frequency=105, score=0.98596911773225),
 KeywordScore(word='엑스', frequency=101, score=0.9847950780631249)]